## <b><font color='darkblue'>Preface</font></b>
([course source](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/5/persistence-and-streaming))

In [1]:
!pip freeze | grep -P '(openai|langchain|langgraph)'

langchain==0.2.6
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.10
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
langgraph==0.1.4
openai==1.28.1


In [2]:
import os
import openai
import re
import httpx
import os
from tavily import TavilyClient
from dotenv import load_dotenv, find_dotenv

from openai import OpenAI
from langchain_openai import ChatOpenAI

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

## <b><font color='darkblue'>Persistence</font></b>
Here we are going to demonstrate the persistence of agent. So it could use chat history to answer question which hidden information which it should refer to the previous chat question.

In [15]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage

In [4]:
tool = TavilySearchResults(max_results=2)

Let's prepare the state of graph which will keep the generated message:

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

Then we use [**SqliteSaver**](SqliteSaver) DB to save the chat history:

In [34]:
memory = SqliteSaver.from_conn_string(":memory:")

Then, below is the implementation of our agent. Here we compile the LangGraph workflow with a [`CheckPointer`](https://langchain-ai.github.io/langgraph/reference/checkpoints/?h=sqlite+saver#checkpoints) to give the agent "memory" by persisting its state. This permits things like:
* Remembering things across multiple interactions
* Interrupting to wait for user input
* Resilience for long-running, error-prone agents
* Time travel retry and branch from a previous checkpoint

In [8]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

Then we could prepare an agent for testing:

In [35]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

### <b><font color='darkgreen'>Question: `What is the weather in Taiwan?`</font></b>

In [36]:
messages = [HumanMessage(content='What is the weather in Taiwan?')]

In [37]:
thread = {"configurable": {"thread_id": "1"}}

We then call [graph.stream](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.stream) with argument `config` to keep the `thread_id` information for futher back tracking.

In [38]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_U6KCkVEkwnuQp4AkwAGwuepY', 'function': {'arguments': '{\n  "query": "current weather in Taiwan"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-83f0b598-6c0c-42bb-90fe-d71bea685e2d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Taiwan'}, 'id': 'call_U6KCkVEkwnuQp4AkwAGwuepY'}], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Taiwan'}, 'id': 'call_U6KCkVEkwnuQp4AkwAGwuepY'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.timeanddate.com/weather/taiwan/taipei/ext\', \'content\': \'Taipei 1

Because we store the chatting history in memory, so we could know below question is referring to the weather of Tokyo.

In [40]:
messages = [HumanMessage(content="What about in Tokyo?")]
thread = {"configurable": {"thread_id": "1"}}
last_event = None
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        last_event = v
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Kldot0N18wuhgs82yzo1fQJE', 'function': {'arguments': '{"query":"current weather in Tokyo"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 1165, 'total_tokens': 1186}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-aad4b30d-d5aa-40c4-9bde-35015dcbe0c1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_Kldot0N18wuhgs82yzo1fQJE'}], usage_metadata={'input_tokens': 1165, 'output_tokens': 21, 'total_tokens': 1186})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_Kldot0N18wuhgs82yzo1fQJE'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.accuweather.com/en/jp/tokyo/226396/january-weat

In [43]:
print(last_event['messages'][-1].content)

The current weather in Tokyo, Japan is as follows:

- **Temperature:** 27.3°C (81.1°F)
- **Condition:** Light rain
- **Wind:** 8.1 mph (13.0 kph) from the south
- **Humidity:** 84%
- **Cloud Cover:** 75%
- **Feels Like:** 29.0°C (84.2°F)
- **Visibility:** 10 km (6 miles)

For more detailed and up-to-date information, you can visit:
- [AccuWeather - Tokyo Weather](https://www.accuweather.com/en/jp/tokyo/226396/january-weather/226396)

Or check other sources such as Weather API or similar weather services.


Now we know both the weather in Taipei and Tokyo. With chatting history, agent should be able to answer below question:

In [44]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
last_event = None
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
        last_event = v

{'messages': [AIMessage(content='Based on the current weather information:\n\n- **Taipei, Taiwan:** 84°F (approximately 29°C)\n- **Tokyo, Japan:** 27.3°C (81.1°F)\n\nTaipei is currently slightly warmer than Tokyo.', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 1822, 'total_tokens': 1872}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4008e3b719', 'finish_reason': 'stop', 'logprobs': None}, id='run-680d63f3-fc4f-4909-b8bc-ffaf8ace11bf-0', usage_metadata={'input_tokens': 1822, 'output_tokens': 50, 'total_tokens': 1872})]}


In [45]:
print(last_event['messages'][-1].content)

Based on the current weather information:

- **Taipei, Taiwan:** 84°F (approximately 29°C)
- **Tokyo, Japan:** 27.3°C (81.1°F)

Taipei is currently slightly warmer than Tokyo.


## <b><font color='darkblue'>Streaming tokens</font></b>

In [47]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [48]:
messages = [HumanMessage(content="What is the weather in German?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


Germany| is| a| country|,| so| the| weather| can| vary| widely| depending| on| the| region|.| Could| you| please| specify| a| particular| city| or| region| in| Germany| for| which| you| would| like| to| know| the| weather|?|

## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearn.ai - AI Agents in LangGraph - Ch2: Build an Agent from Scratch](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/2/build-an-agent-from-scratch)
* [Deeplearn.ai - AI Agents in LangGraph - Ch3: LangGraph components](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/3/langgraph-components)
* [Deeplearn.ai - AI Agents in LangGraph - Ch4: Agentic search tools](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/4/agentic-search-tools)
* [Deeplearn.ai - AI Agents in LangGraph - Ch5: Persistence and streaming](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/5/persistence-and-streaming)
* [Deeplearn.ai - AI Agents in LangGraph - Ch6: Human in the loop](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/6/human-in-the-loop)
* [Langchain doc - How to add chat history](https://python.langchain.com/v0.2/docs/how_to/qa_chat_history_how_to/)
* [YWC 科技筆記 - LangGraph: LangChain Agent 的殺手鐧 (進階)](https://ywctech.net/ml-ai/langchain-langgraph-agent-part2/)